# Shopping Mall

## 상품판매 수량

---

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'NanumGothic'
rcParams['font.size'] = 10

### 파일 로드

In [ ]:
df = pd.read_excel('data/master.xlsx', sheet_name='order',encoding='cp949')
df

In [ ]:
df.columns

In [ ]:
df1 = df[['상품번호','주문일시','배송지','sex','gubun','진행구분']].copy()
df1.head()

### 임직원 제외 & 구매확정

In [ ]:
df1['gubun'].unique()

In [ ]:
df['진행구분'].unique()

In [ ]:
df1 = df1.loc[(df1['gubun']=='일반') & ( (df1['진행구분']=='결제완료')  |
                                         (df1['진행구분']=='구매확정')   |
                                         (df1['진행구분']=='배송완료')   |
                                         (df1['진행구분']=='배송준비중') |
                                         (df1['진행구분']=='배송중')     |
                                         (df1['진행구분']=='주문완료')    )]

In [ ]:
len(df1)

### 성별 결측치 처리

In [ ]:
df1['sex'].value_counts()

In [ ]:
df1['sex'].isnull().sum()

In [ ]:
df1['sex'].fillna('U',inplace=True)

In [ ]:
df1['sex'].replace('-','U', inplace=True)

# apply 활용
#df1['sex'] = df1['sex'].apply(lambda x: 'U' if x == '-' else x)

# loc 활용
#df1.loc[df1['sex'] == '-', 'sex'] = 'U'

In [ ]:
df1['sex'].value_counts()

### 컬럼(지역)추가 from 배송지

In [ ]:
df1['배송지'].isnull().sum()

In [ ]:
def func1(x):
    if not isinstance(x,str): return '기타'
    if '서울'     in x: return '서울'
    if '세종'     in x: return '세종'
    if '제주'     in x: return '제주'
    if '경기'     in x: return '경기'
    if '인천'     in x: return '인천'
    if '대구'     in x: return '대구'
    if '전라북도' in x: return '전북'
    if '테헤란로' in x: return '서울'
    if '죽전'     in x: return '경기'
    if '라도무스' in x: return '기타'
    if '102*301'  in x: return '기타'
    if '호계e편한세상' in x: return '기타'
    return x.split()[0]

In [ ]:
df1['지역'] = df1['배송지'].apply(func1)

In [ ]:
df1['지역'].value_counts()

In [ ]:
df1.head()

### 상품판매 수량 순위(전체)

In [ ]:
gp1 = df1.groupby(['상품번호'])

In [ ]:
df_item1 = gp1.count().reset_index().sort_values(by=['배송지'], ascending=False)
df_item1

In [ ]:
df_item_all = df_item1.rename(columns={'배송지':'판매수량'})[['상품번호','판매수량']]
df_item_all

### 상품판매 수량 순위(지역별)

In [ ]:
gp2 = df1.groupby(['지역','상품번호'])

In [ ]:
df_item2 = gp2.count().reset_index().sort_values(by=['지역','배송지'], ascending=False)
df_item2

In [ ]:
df_item_loc = df_item2.rename(columns={'배송지':'판매수량'})[['지역','상품번호','판매수량']]
df_item_loc

### 상품판매 수량 순위(성별)

In [ ]:
gp3 = df1.groupby(['sex','상품번호'])

In [ ]:
df_item3 = gp3.count().reset_index().sort_values(by=['sex','배송지'], ascending=False)
df_item3

In [ ]:
df_item_sex = df_item3.rename(columns={'배송지':'판매수량'})[['sex','상품번호','판매수량']]
df_item_sex

### 상품판매 수량 순위(요일별)

In [ ]:
# datetime 형식으로 변경
df1['주문일시'] = pd.to_datetime(df1['주문일시'])

In [ ]:
# 요일 컬럼 추가
df1['wday'] = df1['주문일시'].dt.dayofweek    # 0: Monday, 1: Tuesday, ...

In [ ]:
gp4 = df1.groupby(['wday','상품번호'])

In [ ]:
df_item4 = gp4.count().reset_index().sort_values(by=['wday','배송지'], ascending=False)
df_item4

In [ ]:
df_item_wday = df_item4.rename(columns={'배송지':'판매수량'})[['wday','상품번호','판매수량']]
df_item_wday

### 결과 파일 저장 ( 상품판매 수량 순위 - 전체, 지역별, 성별, 요일별)

In [ ]:
df_item_all.to_csv('data/result/item_cnt_all.csv', index=False, encoding='cp949' ) 

In [ ]:
df_item_loc.to_csv('data/result/item_cnt_loc.csv', index=False, encoding='cp949' ) 

In [ ]:
df_item_sex.to_csv('data/result/item_cnt_sex.csv', index=False, encoding='cp949' ) 

In [ ]:
df_item_wday.to_csv('data/result/item_cnt_wday.csv', index=False, encoding='cp949' ) 

### 그래프

In [ ]:
# 상품판매 수량 순위
df_item_all.head(10).plot(x='상품번호', kind='bar')

In [ ]:
# 월요일 상품별 판매 수량 순위
df_item_wday[df_item_wday['wday']==0].head(10).plot(x='상품번호', y='판매수량', kind='bar')
plt.title('월요일 상품별 판매 수량 순위')
plt.show()

In [ ]:
# end of file